# Solana Data Explorer - Delta Lake Tables (DuckDB Edition)

Simple viewer for the most recent data in each Delta Lake table from our DAG pipeline.
Uses DuckDB with Delta Lake support - no Java/Spark required!

## Tables:
- **Bronze**: token_metrics, whale_holders, transaction_history
- **Silver**: tracked_tokens_delta, tracked_whales_delta, wallet_pnl
- **Gold**: smart_traders_delta

In [ ]:
import duckdb
import pandas as pd
from datetime import datetime

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 10)

print("📊 Solana Data Explorer - DuckDB Delta Lake Edition")
print(f"🕐 Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Create DuckDB connection with extensions
print("🦆 Initializing DuckDB with Delta Lake support...")

import os
# Set environment variables to prevent AWS metadata lookup
os.environ['AWS_EC2_METADATA_DISABLED'] = 'true'

conn = duckdb.connect()

# Install and load required extensions
conn.execute("INSTALL delta;")
conn.execute("INSTALL httpfs;")
conn.execute("LOAD delta;")
conn.execute("LOAD httpfs;")

# Method 2: CREATE SECRET (Modern and Recommended)
try:
    conn.execute("""
        CREATE SECRET minio_secret (
            TYPE S3,
            ENDPOINT 'localhost:9000',
            URL_STYLE 'path',
            USE_SSL false,
            KEY_ID 'minioadmin',
            SECRET 'minioadmin123'
        );
    """)
    print("✅ MinIO secret created successfully")
except Exception as e:
    # Handle cases where the secret might already exist
    if "already exists" in str(e):
        print("✅ MinIO secret already exists")
    else:
        print(f"⚠️ Secret creation failed: {e}")
        # Fallback to SET commands
        print("📝 Falling back to SET commands...")
        conn.execute("SET s3_endpoint='localhost:9000';")
        conn.execute("SET s3_use_ssl=false;")
        conn.execute("SET s3_url_style='path';")
        conn.execute("SET s3_access_key_id='minioadmin';")
        conn.execute("SET s3_secret_access_key='minioadmin123';")

print("✅ DuckDB ready with Delta Lake support")
print("📝 Note: Make sure MinIO is running on localhost:9000")

## 🥉 Bronze Layer Tables

In [ ]:
# Bronze Token Metrics
print("📦 BRONZE: Token Metrics (Latest 10 records)")
print("="*70)

try:
    # Query Delta table using delta_scan
    result = conn.execute("""
        SELECT * FROM delta_scan('s3://smart-trader/bronze/token_metrics')
        ORDER BY _delta_timestamp DESC
        LIMIT 10
    """).df()
    
    # Get total count
    count_result = conn.execute("""
        SELECT COUNT(*) as total FROM delta_scan('s3://smart-trader/bronze/token_metrics')
    """).fetchone()
    
    print(f"Total records: {count_result[0]:,}")
    print(f"\nLatest 10 tokens:")
    display_cols = ['token_address', 'symbol', 'liquidity', 'price', 'processing_date', '_delta_timestamp']
    available_cols = [col for col in display_cols if col in result.columns]
    print(result[available_cols] if available_cols else result)
    
except Exception as e:
    print(f"❌ Error loading bronze tokens: {e}")

In [ ]:
# Bronze Whale Holders
print("\n📦 BRONZE: Whale Holders (Latest 10 records)")
print("="*70)

try:
    result = conn.execute("""
        SELECT * FROM delta_scan('s3://smart-trader/bronze/whale_holders')
        ORDER BY _delta_timestamp DESC
        LIMIT 10
    """).df()
    
    count_result = conn.execute("""
        SELECT COUNT(*) as total FROM delta_scan('s3://smart-trader/bronze/whale_holders')
    """).fetchone()
    
    print(f"Total records: {count_result[0]:,}")
    print(f"\nLatest 10 whale holders:")
    display_cols = ['wallet_address', 'token_symbol', 'rank', 'holdings_amount', 'holdings_value_usd', '_delta_timestamp']
    available_cols = [col for col in display_cols if col in result.columns]
    print(result[available_cols] if available_cols else result)
    
except Exception as e:
    print(f"❌ Error loading bronze whales: {e}")

In [ ]:
# Bronze Transaction History
print("\n📦 BRONZE: Transaction History (Latest 10 records)")
print("="*70)

try:
    result = conn.execute("""
        SELECT * FROM delta_scan('s3://smart-trader/bronze/transaction_history')
        ORDER BY timestamp DESC
        LIMIT 10
    """).df()
    
    count_result = conn.execute("""
        SELECT COUNT(*) as total FROM delta_scan('s3://smart-trader/bronze/transaction_history')
    """).fetchone()
    
    print(f"Total records: {count_result[0]:,}")
    print(f"\nLatest 10 transactions:")
    display_cols = ['wallet_address', 'timestamp', 'base_symbol', 'quote_symbol', 'base_type_swap', 'quote_type_swap']
    available_cols = [col for col in display_cols if col in result.columns]
    print(result[available_cols] if available_cols else result)
    
except Exception as e:
    print(f"❌ Error loading bronze transactions: {e}")

## 🥈 Silver Layer Tables

In [ ]:
# Silver Tracked Tokens
print("💎 SILVER: Tracked Tokens (Latest 10 records)")
print("="*70)

try:
    result = conn.execute("""
        SELECT * FROM delta_scan('s3://smart-trader/silver/tracked_tokens_delta')
        ORDER BY silver_created_at DESC
        LIMIT 10
    """).df()
    
    count_result = conn.execute("""
        SELECT COUNT(*) as total FROM delta_scan('s3://smart-trader/silver/tracked_tokens_delta')
    """).fetchone()
    
    print(f"Total records: {count_result[0]:,}")
    print(f"\nLatest 10 tracked tokens:")
    display_cols = ['token_address', 'symbol', 'liquidity', 'liquidity_tier', 'whale_fetch_status', 'silver_created_at']
    available_cols = [col for col in display_cols if col in result.columns]
    print(result[available_cols] if available_cols else result)
    
except Exception as e:
    print(f"❌ Error loading silver tokens: {e}")

In [ ]:
# Silver Tracked Whales
print("\n💎 SILVER: Tracked Whales (Latest 10 records)")
print("="*70)

try:
    result = conn.execute("""
        SELECT * FROM delta_scan('s3://smart-trader/silver/tracked_whales_delta')
        ORDER BY silver_created_at DESC
        LIMIT 10
    """).df()
    
    count_result = conn.execute("""
        SELECT COUNT(*) as total FROM delta_scan('s3://smart-trader/silver/tracked_whales_delta')
    """).fetchone()
    
    print(f"Total records: {count_result[0]:,}")
    print(f"\nLatest 10 tracked whales:")
    display_cols = ['whale_id', 'wallet_address', 'token_symbol', 'whale_tier', 'processing_status', 'silver_created_at']
    available_cols = [col for col in display_cols if col in result.columns]
    print(result[available_cols] if available_cols else result)
    
except Exception as e:
    print(f"❌ Error loading silver whales: {e}")

In [ ]:
# Silver Wallet PnL
print("\n💎 SILVER: Wallet PnL (Latest 10 records)")
print("="*70)

try:
    result = conn.execute("""
        SELECT * FROM delta_scan('s3://smart-trader/silver/wallet_pnl')
        WHERE token_address = 'ALL_TOKENS'
        ORDER BY processed_at DESC
        LIMIT 10
    """).df()
    
    count_result = conn.execute("""
        SELECT COUNT(*) as total FROM delta_scan('s3://smart-trader/silver/wallet_pnl')
        WHERE token_address = 'ALL_TOKENS'
    """).fetchone()
    
    print(f"Total portfolio records: {count_result[0]:,}")
    print(f"\nLatest 10 wallet PnL records:")
    display_cols = ['wallet_address', 'total_pnl', 'portfolio_roi', 'win_rate', 'trade_count', 'processed_at']
    available_cols = [col for col in display_cols if col in result.columns]
    print(result[available_cols] if available_cols else result)
    
except Exception as e:
    print(f"❌ Error loading silver PnL: {e}")

## 🥇 Gold Layer Tables

In [ ]:
# Gold Smart Traders
print("🏆 GOLD: Smart Traders (Latest 10 records)")
print("="*70)

try:
    result = conn.execute("""
        SELECT * FROM delta_scan('s3://smart-trader/gold/smart_traders_delta')
        ORDER BY total_pnl DESC
        LIMIT 10
    """).df()
    
    count_result = conn.execute("""
        SELECT COUNT(*) as total FROM delta_scan('s3://smart-trader/gold/smart_traders_delta')
    """).fetchone()
    
    # Get tier breakdown
    tier_counts = conn.execute("""
        SELECT performance_tier, COUNT(*) as count 
        FROM delta_scan('s3://smart-trader/gold/smart_traders_delta')
        GROUP BY performance_tier
        ORDER BY count DESC
    """).df()
    
    print(f"Total smart traders: {count_result[0]:,}")
    
    print(f"\nPerformance tiers:")
    for _, row in tier_counts.iterrows():
        print(f"  {row['performance_tier']}: {row['count']:,}")
    
    print(f"\nTop 10 smart traders by PnL:")
    display_cols = ['wallet_address', 'total_pnl', 'roi', 'win_rate', 'performance_tier', 'gold_processed_at']
    available_cols = [col for col in display_cols if col in result.columns]
    print(result[available_cols] if available_cols else result)
    
except Exception as e:
    print(f"❌ Error loading gold traders: {e}")

## 📊 Pipeline Summary

In [ ]:
# Pipeline Summary
print("\n📊 PIPELINE SUMMARY")
print("="*70)

# Collect all table stats
table_stats = []

tables = [
    ("Bronze Token Metrics", "s3://smart-trader/bronze/token_metrics"),
    ("Bronze Whale Holders", "s3://smart-trader/bronze/whale_holders"),
    ("Bronze Transactions", "s3://smart-trader/bronze/transaction_history"),
    ("Silver Tracked Tokens", "s3://smart-trader/silver/tracked_tokens_delta"),
    ("Silver Tracked Whales", "s3://smart-trader/silver/tracked_whales_delta"),
    ("Silver Wallet PnL", "s3://smart-trader/silver/wallet_pnl"),
    ("Gold Smart Traders", "s3://smart-trader/gold/smart_traders_delta")
]

for table_name, table_path in tables:
    try:
        count_result = conn.execute(f"""
            SELECT COUNT(*) as total FROM delta_scan('{table_path}')
        """).fetchone()
        
        table_stats.append({
            "Table": table_name,
            "Records": f"{count_result[0]:,}",
            "Status": "✅ Active"
        })
    except Exception as e:
        table_stats.append({
            "Table": table_name,
            "Records": "0",
            "Status": "❌ Missing"
        })

# Display summary
summary_df = pd.DataFrame(table_stats)
print(summary_df.to_string(index=False))

print(f"\n✅ Data exploration complete!")
print(f"🕐 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Close DuckDB connection
conn.close()
print("🦆 DuckDB connection closed")